In [13]:
import tensorflow as tf

x = tf.Variable(3, name = "x")
y = tf.Variable(4, name = "y")
# c = tf.constant([1, 2, 3], name = "c")

def f (x, y, c):
    return x+y+c

print(f(x, y, [3, 3, 3]))

print(f(x, y, [3, 3, 3]).numpy())



tf.Tensor([10 10 10], shape=(3,), dtype=int32)
[10 10 10]


\begin{aligned}
& \min
& & \sum_{j=1}^n c_j x_j \\
& \text{s.t.}
& & \sum_{j=1}^n a_{ij} x_j \geq k, \; i = 1, \cdots, m \\
& & &  \; x_j \in \{0,1\}, \;  j = 1, \cdots, n 
\end{aligned}

\begin{equation*}
\begin{aligned}
& \min
& & \sum_{j=1}^n c_j x_j \\
& \; \text{s.t.}
& & \sum_{j=1}^n a_{ij} x_j \geq k, \; i = 1, \cdots, m \\
& & &  \; x_j \in \{0,1\}, \;  j = 1, \cdots, n 
\end{aligned}
\end{equation*}

\begin{equation}
\min \sum_{j=1}^n x^2 + y^2 = z^2
\end{equation}

In [21]:
d = {1111:1, 123:1, 234:1, 456:3, 345:8}
x = min(d, key=d.get)
print(x)

1111


In [46]:
dataIndx

array([False, False, False, ...,  True,  True,  True])

# __<center>ECE421 - Winter 2020__
# __<center> Assignment 1:__
# **<center>Linear and Logistic Regression**